# Run the Bayesian Online Change-Point Detector vs. the Benchmarks

In [4]:
# Load imports
import numpy as np 
import pandas as pd 

from models.Simulator import Simulator
from models.stratagy import *

**Load Data**

In [9]:
returns = pd.read_csv("returns", sep=",", index_col="Date")
returns.tail()

,AAPL,ADBE,AMZN,BAC,CSCO,CVX,DIS,GOOGL,HD,INTC,JNJ,JPM,KO,MRK,MSFT,NVDA,PEP,PFE,PG,T,VZ,WMT,SAP.DE,BMW.DE,BAS.DE,AIR.PA,SHEL.L,VOD.L,HSBA.L,AZN.L,BP.L,7203.T,6861.T,9984.T,6954.T,8411.T,CS.PA,SAN.PA,BNP.PA,MC.PA,OR.PA,ULVR.L,BARC.L,RIO.L,TLT,IEF,SHY,BWX,PHAU.L,GLD,XLE,BTC-USD
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-05-23,-0.030711,-0.016156,-0.010443,-0.002543,-0.003954,0.009197,-0.012769,-0.014145,-0.009385,-0.024632,0.002160,0.000153,0.008817,-0.003346,-0.010342,-0.011662,-0.006012,0.012079,0.005017,0.006953,0.004396,0.004265,-0.023184,-0.042173,-0.028462,-0.029227,-0.008217,-0.023318,-0.013477,0.005972,-0.001670,0.002051,-0.000424,0.000984,0.000972,0.007416,-0.018903,-0.018054,-0.026016,-0.017019,-0.022433,-0.003601,-0.010295,-0.005342,0.001657,0.002884,0.000606,0.012717,0.019181,0.021670,0.003176,-0.040063
2025-05-27,0.024984,0.013183,0.024718,0.023337,0.010090,0.009403,0.023776,0.025956,0.020980,0.024632,0.010561,0.017415,0.000139,0.000129,0.023078,0.031563,0.015573,0.012359,0.011390,-0.001095,0.004147,0.012789,0.014068,-0.003321,0.013345,0.013659,0.013059,0.007343,0.000555,0.006757,0.012919,0.005547,0.000290,0.020548,0.003085,0.011737,0.001861,0.001485,0.003964,-0.001834,-0.000362,0.005570,0.016906,-0.011959,0.013860,0.004151,0.000364,-0.001744,-0.018692,-0.017094,0.008622,-0.004081
2025-05-28,0.001048,-0.002108,-0.006330,-0.003625,-0.006452,-0.013219,-0.007504,-0.003128,-0.006338,-0.008798,-0.005365,-0.006808,-0.008816,-0.018471,-0.007255,-0.005105,-0.005343,-0.018381,-0.002387,0.004008,-0.008542,-0.003490,-0.013408,0.037743,-0.012720,-0.003660,-0.004445,-0.009525,-0.006140,-0.013223,-0.008843,0.007388,0.000403,0.011766,-0.003673,0.016154,-0.010591,-0.006464,-0.015431,-0.000897,-0.023310,-0.010220,-0.016561,-0.014447,-0.004325,-0.002765,-0.000727,-0.003936,-0.000342,-0.002269,-0.012778,-0.010999
2025-05-29,-0.002348,0.002737,0.004776,0.004077,-0.004589,0.013799,0.004473,-0.002905,0.000625,-0.005908,0.007516,0.003334,0.004767,0.003015,0.002882,0.031974,0.009521,0.011581,0.007145,-0.004373,0.004626,-0.001441,-0.012397,-0.025091,-0.006815,-0.007809,-0.007508,-0.008675,0.000833,-0.004921,-0.006774,0.048882,0.026966,0.024221,0.036107,0.016997,0.000287,-0.016206,-0.005904,-0.014138,-0.010119,-0.006717,0.000386,-0.003832,0.009212,0.004145,0.001091,0.006987,0.006364,0.005907,0.007443,-0.020246
2025-05-30,0.004491,0.004177,-0.003360,-0.002490,-0.000159,-0.008813,0.009064,-0.000698,0.000000,-0.035180,0.010557,-0.001401,0.008496,0.005743,0.003656,-0.029603,-0.003569,0.001704,0.007859,0.015223,0.014435,0.016546,0.025858,0.007944,0.007623,0.009841,0.008286,0.005275,0.007078,0.035512,0.009891,-0.000714,-0.040375,-0.052402,-0.045057,-0.014500,0.014003,-0.036797,0.016825,0.006545,0.029860,0.011195,0.009452,-0.008300,0.001508,0.002965,0.001211,-0.001742,-0.008875,-0.006599,-0.008914,-0.015677


### Start with running all the stratagies